## DataTransformer:
A notebook to help with the transformation of `vehicle` data into the a merged format and possibly a parquet format


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/road-accidents-2019-2023/vehicles-2020.csv
/kaggle/input/road-accidents-2019-2023/char-2019.csv
/kaggle/input/road-accidents-2019-2023/char-2021.csv
/kaggle/input/road-accidents-2019-2023/place-2022.csv
/kaggle/input/road-accidents-2019-2023/users-2023.csv
/kaggle/input/road-accidents-2019-2023/users-2021.csv
/kaggle/input/road-accidents-2019-2023/vehicles-2021.csv
/kaggle/input/road-accidents-2019-2023/char-2020.csv
/kaggle/input/road-accidents-2019-2023/place-2019.csv
/kaggle/input/road-accidents-2019-2023/users-2019.csv
/kaggle/input/road-accidents-2019-2023/place-2021.csv
/kaggle/input/road-accidents-2019-2023/users-2020.csv
/kaggle/input/road-accidents-2019-2023/char-2023.csv
/kaggle/input/road-accidents-2019-2023/users-2022.csv
/kaggle/input/road-accidents-2019-2023/vehicles-2022.csv
/kaggle/input/road-accidents-2019-2023/vehicles-2019.csv
/kaggle/input/road-accidents-2019-2023/place-2020.csv
/kaggle/input/road-accidents-2019-2023/place-2023.csv
/kaggle/input/road-a

In [5]:
%%time
## all imports
import polars as pl
import pyarrow.parquet as pq
import dask.dataframe as dd
import os
import shutil
import json
from enum import Enum
from datetime import datetime
from ydata_profiling import ProfileReport
from pathlib import Path

#For excel stuff
import openpyxl
from openpyxl.drawing.image import Image


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Concurrency
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import time

#  Warnings
import warnings
warnings.filterwarnings('ignore')

# Set random state
random_state = 42
# Set figure size
plt.rcParams["figure.figsize"] = (20, 20)


## Set the static file locactions
filepaths = {'vehicles':
    {
    2019: '/kaggle/input/road-accidents-2019-2023/vehicles-2019.csv',
    2020: '/kaggle/input/road-accidents-2019-2023/vehicles-2020.csv',
    2021: '/kaggle/input/road-accidents-2019-2023/vehicles-2021.csv',
    2022: '/kaggle/input/road-accidents-2019-2023/vehicles-2022.csv',
    2023: '/kaggle/input/road-accidents-2019-2023/vehicles-2023.csv',
    },
    'users':{
        2019: '/kaggle/input/road-accidents-2019-2023/users-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/users-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/users-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/users-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/users-2023.csv'
    },
    'places': {
        2019: '/kaggle/input/road-accidents-2019-2023/place-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/place-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/place-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/place-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/place-2023.csv'
    },
    'characteristics':{
        2019: '/kaggle/input/road-accidents-2019-2023/char-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/char-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/char-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/char-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/char-2023.csv'
    }
             
}

CPU times: user 137 µs, sys: 0 ns, total: 137 µs
Wall time: 144 µs


In [6]:
class ExtensionMethods:
    @staticmethod
    def generate_filename(filename=None,extension=None):
        current_datetime = datetime.now()
        f = current_datetime.strftime("%Y_%m_%d_%H%M")
        if (filename is None) or (extension is None):
            return str(f)
        else:
            stitched_f = str(filename)+"_"+str(f)+"."+str(extension)
            return str(stitched_f)

    @staticmethod
    def get_file_name_without_extension(filename):
        if filename == None:
            return "Provide a file"
        return Path(filename).stem
        
        

In [11]:
df1 = pd.read_csv(filepaths['vehicles'][2023], sep=';')
df2 = pd.read_csv(filepaths['users'][2023], sep=';')
df3 = pd.read_csv(filepaths['characteristics'][2023], sep=';')
df4 = pd.read_csv(filepaths['places'][2023], sep=';')

df1 = df1.head()
df2 = df2.head()
df3 = df3.head()
df4 = df4.head()

In [12]:
#df1 = pd.DataFrame({'Num_Acc': "X11", 'A': ['A1', 'A2', 'A3'],'G':[1,2,3]})
#df2 = pd.DataFrame({'Num_Acc': "X11", 'B': ['B2', 'B3', 'B4'], "F":[2,3,4]})
#df3 = pd.DataFrame({'Num_Acc': "X11", 'C': ['C3', 'C4', 'C5'],"K":['g','gg']})
#df4 = pd.DataFrame({'Num_Acc': "X11", 'D': ['D1', 'D3', 'D5'],"I":"XXX"})


dataframes = [df1,df2,df3,df4]



print(f"Vehicles: {df1['Num_Acc'].value_counts()}")
print(df2['Num_Acc'].value_counts())
print(df3['Num_Acc'].value_counts())
print(df4['Num_Acc'].value_counts())


merged_df = dataframes[0] # take the first
for df in dataframes[1:]:
    merged_df = pd.merge(merged_df,df) ## I guess the default takes all three values



Vehicles: Num_Acc
202300000003    2
202300000001    1
202300000002    1
202300000004    1
Name: count, dtype: int64
Num_Acc
202300000002    2
202300000003    2
202300000001    1
Name: count, dtype: int64
Num_Acc
202300000001    1
202300000002    1
202300000003    1
202300000004    1
202300000005    1
Name: count, dtype: int64
Num_Acc
202300000001    2
202300000003    2
202300000002    1
Name: count, dtype: int64


In [13]:
with pd.option_context('display.max_columns', None):
    print(merged_df)

        Num_Acc  id_vehicule num_veh  senc  catv  obs  obsm  choc  manv  \
0  202300000001  155 680 557     A01     1    30    0     0     5     1   
1  202300000001  155 680 557     A01     1    30    0     0     5     1   
2  202300000002  155 680 556     A01     2     7    0     1     1     1   
3  202300000002  155 680 556     A01     2     7    0     1     1     1   
4  202300000003  155 680 554     B01     1     2    0     2     1    16   
5  202300000003  155 680 554     B01     1     2    0     2     1    16   
6  202300000003  155 680 555     A01     2     7    0     2     2    15   
7  202300000003  155 680 555     A01     2     7    0     2     2    15   

   motor  occutc    id_usager  place  catu  grav  sexe  an_nais  trajet  \
0      1     NaN  203 851 184      1     1     4     1   1978.0       5   
1      1     NaN  203 851 184      1     1     4     1   1978.0       5   
2      1     NaN  203 851 182      1     1     1     2   1997.0       9   
3      1     NaN  203 85

In [15]:
%%time
years = [2019,2020,2021,2022,2023]


### Just testing



def create_dataframe_list(year):
    dataframe = []
    for key,value in filepaths.items():
        filename = filepaths[key][year]
        _df = pd.read_csv(filename,sep=';')
        dataframe.append(_df)
    return dataframe
    

def _merge(year=None):
    _dataframe = create_dataframe_list(year)
    merged_df = _dataframe[0]
    for df in _dataframe[1:]:
        merged_df = pd.merge(merged_df,df)
    return merged_df


def _concat_all_merged():
    _mega_concat= {}
    _dfs = []
    for year in years:
        print(f"\n Doing year {year}")
        _mega_concat[year] = _merge(year)
    for key,value in _mega_concat.items():
        print(key)
        value['csv_info']=key
        _dfs.append(value)
    _concated = pd.concat(_dfs)
    return _concated
        

a = _concat_all_merged()
a


 Doing year 2019

 Doing year 2020

 Doing year 2021

 Doing year 2022

 Doing year 2023
2019
2020
2021
2022
2023
CPU times: user 5.94 s, sys: 813 ms, total: 6.75 s
Wall time: 7.72 s


,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,...,com,agg,int,atm,col,adr,lat,long,csv_info,id_usager
0,201900000001,138 306 524,B01,2,7,0,2,5,23,1,...,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200",2019,NaN
1,201900000001,138 306 524,B01,2,7,0,2,5,23,1,...,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200",2019,NaN
2,201900000001,138 306 525,A01,2,17,1,0,3,11,1,...,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200",2019,NaN
3,201900000002,138 306 523,A01,1,7,4,0,1,0,1,...,93066,1,1,1,6,AUTOROUTE A1,"48,9307000","2,3688000",2019,NaN
4,201900000003,138 306 520,A01,1,7,0,2,1,2,1,...,92036,1,1,1,4,AUTOROUTE A86,"48,9358718","2,3191744",2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163678,202300054821,155 583 346,A01,1,30,0,2,3,9,1,...,97302,1,6,1,3,ROUTE NATIONALE 1,"4,89713000","-52,32854000",2023,203 720 720
163679,202300054821,155 583 347,B01,1,7,0,2,2,9,1,...,97302,1,6,1,3,ROUTE NATIONALE 1,"4,89713000","-52,32854000",2023,203 720 721
163680,202300054822,155 583 344,A01,2,7,0,2,7,22,1,...,69387,2,1,6,3,Boulevard Yves Farge,"45,73306000","4,82540000",2023,203 720 717
163681,202300054822,155 583 344,A01,2,7,0,2,7,22,1,...,69387,2,1,6,3,Boulevard Yves Farge,"45,73306000","4,82540000",2023,203 720 718


In [60]:
class DataTransformer:
    def __init__(self,file_dict,key=None):
        if file_dict is None or not bool(file_dict): ## checks if none or empty
            raise "Please provide a file path dictionary"
        self.file_dict = file_dict
        if key is None:
            #Do it for all keys
            self.key = [key for key in self.file_dict] ## should get keys list
        else:
            self.key = [key] ## assume list for all

        self.dirpath = self.create_dir()
        
    def create_dir(self):
        # Dic as we assume its a list
        dic ={}
        for k in self.key:
            dirpath =  os.path.join(os.getcwd(),k)
            if not os.path.exists(dirpath):
                os.makedirs(dirpath)
            dic[k] = dirpath
        return dic        
        
    def _concat(self, key='vehicles'): ## this still remains the same , except we want to do it for keys =='vehicles'
        dfs = []
        for year in self.file_dict[key]:
            filename = self.file_dict[key][year]
            _df = pd.read_csv(filename,sep=';')
            _df['csv_info'] = ExtensionMethods.get_file_name_without_extension(filename)
            dfs.append(_df)
        _concated = pd.concat(dfs)
        return _concated

    def _concat_all(self): ## should just call the above for all the keys
        _mega_concat = {}
        for key in self.key:
            _mega_concat[key] = self._concat(key)
        return _mega_concat ## returns a dict with {key:Concated_Dataframe} 
        

    def create_csv(self):
        vehicles_concated = self._concat_all() ## this is a dict
        for key,value in vehicles_concated.items():
            filepath = os.path.join(self.dirpath[key], ExtensionMethods.generate_filename(f"{key}-merged", "csv"))
            value.to_csv(filepath, index=False)

    def create_parquet(self):
        vehicles_concated = self._concat_all()
        vehicles_concated = vehicles_concated.compute()
        filepath = os.path.join(self.dirpath, ExtensionMethods.generate_filename(f"{self.key}-merged", "parquet"))
        vehicles_concated.to_parquet(filepath, engine='pyarrow', index=False)

In [61]:
dtransformer = DataTransformer(file_dict=filepaths)
dtransformer.create_csv()

vehicles
{2019: '/kaggle/input/road-accidents-2019-2023/vehicles-2019.csv', 2020: '/kaggle/input/road-accidents-2019-2023/vehicles-2020.csv', 2021: '/kaggle/input/road-accidents-2019-2023/vehicles-2021.csv', 2022: '/kaggle/input/road-accidents-2019-2023/vehicles-2022.csv', 2023: '/kaggle/input/road-accidents-2019-2023/vehicles-2023.csv'}
users
{2019: '/kaggle/input/road-accidents-2019-2023/users-2019.csv', 2020: '/kaggle/input/road-accidents-2019-2023/users-2020.csv', 2021: '/kaggle/input/road-accidents-2019-2023/users-2021.csv', 2022: '/kaggle/input/road-accidents-2019-2023/users-2022.csv', 2023: '/kaggle/input/road-accidents-2019-2023/users-2023.csv'}
places
{2019: '/kaggle/input/road-accidents-2019-2023/place-2019.csv', 2020: '/kaggle/input/road-accidents-2019-2023/place-2020.csv', 2021: '/kaggle/input/road-accidents-2019-2023/place-2021.csv', 2022: '/kaggle/input/road-accidents-2019-2023/place-2022.csv', 2023: '/kaggle/input/road-accidents-2019-2023/place-2023.csv'}
characteristics